In [112]:
import pyspark as sp
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
import pyspark.ml as ml
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('BigMartSales').getOrCreate()

In [79]:
spark

In [80]:
df = spark.read.csv('data/cleaned_train.csv', header=True, inferSchema=True)

In [115]:
# import pandas as pd 
# from pyspark.ml.regression import DecisionTreeRegressor

# df_stats = pd.read_csv('constants/stats.csv', index_col='stat')
# Z_SCORE = df_stats.loc['z_score'].values[0]
# STD = df_stats.loc['std'].values[0]

# quick_fit = spark.read.csv('constants/quick_fit.csv', header=True, inferSchema=True).drop('_c0')
# toVector = ['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 'outletAge', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales']
# featuresVector = VectorAssembler(inputCols=toVector,outputCol="featuresVector")
# quick_fit = featuresVector.transform(quick_fit)
# quick_fit = quick_fit.select('featuresVector', 'Item_Outlet_Sales')
# model = DecisionTreeRegressor(featuresCol='featuresVector', labelCol='Item_Outlet_Sales', maxBins=2048)
# model = model.fit(quick_fit)

In [116]:
# def getInterval(pred):
#     return (pred - Z_SCORE*STD, pred + Z_SCORE*STD)

In [129]:
# from pyspark.ml.functions import array_to_vector
# params = str([100, 23, 0, 5, 580, 15, 1, 2, 0])
# item = array_to_vector(params)

In [130]:
# item

Column<'UDF([100, 23, 0, 5, 580, 15, 1, 2, 0])'>

In [131]:
# prediction = model.transform(item)
# print(f"Interval for this item is {getInterval(prediction)}")

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [111]:
df.show()

+--------------------+-----------------+
|      featuresVector|Item_Outlet_Sales|
+--------------------+-----------------+
|[9.3,0.0,0.016047...|         3735.138|
|[5.92,1.0,0.01927...|         443.4228|
|[17.5,0.0,0.01676...|          2097.27|
|[19.2,1.0,0.0,6.0...|           732.38|
|[8.93,0.0,0.0,9.0...|         994.7052|
|[10.395,1.0,0.0,0...|         556.6088|
|[13.65,1.0,0.0127...|         343.5528|
|[19.0,0.0,0.12746...|        4022.7636|
|[16.2,1.0,0.01668...|        1076.5986|
|[19.2,1.0,0.09444...|         4710.535|
|[11.8,0.0,0.0,6.0...|        1516.0266|
|[18.5,1.0,0.04546...|         2187.153|
|[15.1,1.0,0.10001...|        1589.2646|
|[17.6,1.0,0.04725...|        2145.2076|
|[16.35,0.0,0.0680...|         1977.426|
|[9.0,1.0,0.069088...|        1547.3192|
|[11.8,0.0,0.00859...|        1621.8888|
|[9.0,1.0,0.069196...|         718.3982|
|[8.26,0.0,0.03423...|         2303.668|
|[13.35,0.0,0.1024...|        2748.4224|
+--------------------+-----------------+
only showing top

In [81]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+---------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|outletAge|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+---------+-----------+--------------------+-----------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|       19|     Medium|                   1|         S1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|        9|     Medium|                   3|         S2|         443.4228|
|          FDN15|       17.5|         Low Fat|    0.016760075|                Me

In [82]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- outletAge: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: integer (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)



In [83]:
df.describe().show()

+-------+---------------+------------------+----------------+-------------------+-------------+-----------------+-----------------+------------------+-----------+--------------------+-----------+------------------+
|summary|Item_Identifier|       Item_Weight|Item_Fat_Content|    Item_Visibility|    Item_Type|         Item_MRP|Outlet_Identifier|         outletAge|Outlet_Size|Outlet_Location_Type|Outlet_Type| Item_Outlet_Sales|
+-------+---------------+------------------+----------------+-------------------+-------------+-----------------+-----------------+------------------+-----------+--------------------+-----------+------------------+
|  count|           8523|              8523|            8523|               8523|         8523|             8523|             8523|              8523|       8523|                8523|       8523|              8523|
|   mean|           NULL|12.875361375103129|            NULL|0.06613202877895127|         NULL|140.9927819781768|             NULL|20.168133

In [84]:
df.drop('Item_Identifier')

DataFrame[Item_Weight: double, Item_Fat_Content: string, Item_Visibility: double, Item_Type: string, Item_MRP: double, Outlet_Identifier: string, outletAge: int, Outlet_Size: string, Outlet_Location_Type: int, Outlet_Type: string, Item_Outlet_Sales: double]

In [85]:
to_encode = {'Item_Fat_Content': 'isLF',
             'Item_Type': 'itemTypeID',
             'Outlet_Size': 'outletSize',
             'Outlet_Type': 'outletType'}

for column, new_column in to_encode.items():
    indexer = StringIndexer(inputCol=column, outputCol=new_column)
    df = indexer.fit(df).transform(df)

In [86]:
df = df.drop('Item_Identifier').drop('Item_Fat_Content').drop('Item_Type').drop('Outlet_Size').drop('Outlet_Identifier').drop('Outlet_Type')

In [87]:
df.show()

+-----------+---------------+--------+---------+--------------------+-----------------+----+----------+----------+----------+
|Item_Weight|Item_Visibility|Item_MRP|outletAge|Outlet_Location_Type|Item_Outlet_Sales|isLF|itemTypeID|outletSize|outletType|
+-----------+---------------+--------+---------+--------------------+-----------------+----+----------+----------+----------+
|        9.3|    0.016047301|249.8092|       19|                   1|         3735.138| 0.0|       4.0|       0.0|       0.0|
|       5.92|    0.019278216| 48.2692|        9|                   3|         443.4228| 1.0|       8.0|       0.0|       3.0|
|       17.5|    0.016760075| 141.618|       19|                   1|          2097.27| 0.0|       9.0|       0.0|       0.0|
|       19.2|            0.0| 182.095|       20|                   3|           732.38| 1.0|       0.0|       1.0|       1.0|
|       8.93|            0.0| 53.8614|       31|                   3|         994.7052| 0.0|       2.0|       3.0|    

In [88]:
df.printSchema()

root
 |-- Item_Weight: double (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- outletAge: integer (nullable = true)
 |-- Outlet_Location_Type: integer (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)
 |-- isLF: double (nullable = false)
 |-- itemTypeID: double (nullable = false)
 |-- outletSize: double (nullable = false)
 |-- outletType: double (nullable = false)



In [89]:
df.show()

+-----------+---------------+--------+---------+--------------------+-----------------+----+----------+----------+----------+
|Item_Weight|Item_Visibility|Item_MRP|outletAge|Outlet_Location_Type|Item_Outlet_Sales|isLF|itemTypeID|outletSize|outletType|
+-----------+---------------+--------+---------+--------------------+-----------------+----+----------+----------+----------+
|        9.3|    0.016047301|249.8092|       19|                   1|         3735.138| 0.0|       4.0|       0.0|       0.0|
|       5.92|    0.019278216| 48.2692|        9|                   3|         443.4228| 1.0|       8.0|       0.0|       3.0|
|       17.5|    0.016760075| 141.618|       19|                   1|          2097.27| 0.0|       9.0|       0.0|       0.0|
|       19.2|            0.0| 182.095|       20|                   3|           732.38| 1.0|       0.0|       1.0|       1.0|
|       8.93|            0.0| 53.8614|       31|                   3|         994.7052| 0.0|       2.0|       3.0|    

In [90]:
independent_variables = ["Item_Weight", "isLF", "Item_Visibility", "itemTypeID", "Item_MRP", "outletAge", "outletSize", "Outlet_Location_Type", "outletType"]
featuresVector = VectorAssembler(inputCols=independent_variables,outputCol="featuresVector")
output = featuresVector.transform(df)

In [91]:
output.show()

+-----------+---------------+--------+---------+--------------------+-----------------+----+----------+----------+----------+--------------------+
|Item_Weight|Item_Visibility|Item_MRP|outletAge|Outlet_Location_Type|Item_Outlet_Sales|isLF|itemTypeID|outletSize|outletType|      featuresVector|
+-----------+---------------+--------+---------+--------------------+-----------------+----+----------+----------+----------+--------------------+
|        9.3|    0.016047301|249.8092|       19|                   1|         3735.138| 0.0|       4.0|       0.0|       0.0|[9.3,0.0,0.016047...|
|       5.92|    0.019278216| 48.2692|        9|                   3|         443.4228| 1.0|       8.0|       0.0|       3.0|[5.92,1.0,0.01927...|
|       17.5|    0.016760075| 141.618|       19|                   1|          2097.27| 0.0|       9.0|       0.0|       0.0|[17.5,0.0,0.01676...|
|       19.2|            0.0| 182.095|       20|                   3|           732.38| 1.0|       0.0|       1.0|    

In [92]:
for col in independent_variables:
    output = output.drop(col)
output = output.withColumnRenamed('Item_Outlet_Sales', 'totalSales')

In [93]:
output.show()

+----------+--------------------+
|totalSales|      featuresVector|
+----------+--------------------+
|  3735.138|[9.3,0.0,0.016047...|
|  443.4228|[5.92,1.0,0.01927...|
|   2097.27|[17.5,0.0,0.01676...|
|    732.38|[19.2,1.0,0.0,0.0...|
|  994.7052|[8.93,0.0,0.0,2.0...|
|  556.6088|[10.395,1.0,0.0,6...|
|  343.5528|[13.65,1.0,0.0127...|
| 4022.7636|[19.0,0.0,0.12746...|
| 1076.5986|[16.2,1.0,0.01668...|
|  4710.535|[19.2,1.0,0.09444...|
| 1516.0266|(9,[0,4,5,7],[11....|
|  2187.153|[18.5,1.0,0.04546...|
| 1589.2646|[15.1,1.0,0.10001...|
| 2145.2076|[17.6,1.0,0.04725...|
|  1977.426|[16.35,0.0,0.0680...|
| 1547.3192|[9.0,1.0,0.069088...|
| 1621.8888|[11.8,0.0,0.00859...|
|  718.3982|[9.0,1.0,0.069196...|
|  2303.668|[8.26,0.0,0.03423...|
| 2748.4224|[13.35,0.0,0.1024...|
+----------+--------------------+
only showing top 20 rows



In [94]:
train,test = output.randomSplit([0.8, 0.2])

In [95]:
train.toPandas().to_csv('constants/quick_fit.csv')

In [96]:
def autoTest(report, label, pred):
    metrics = {"mae": 0,
               'mse': 0,
               'rmse': 0,
               'r2': 0,
               'var': 0}
    for metric in metrics.keys():
        evaluator = ml.evaluation.RegressionEvaluator(labelCol=label, predictionCol=pred, metricName=metric)
        metrics[metric] = evaluator.evaluate(report)
    print(*[f'{met}: {result}' for met, result in metrics.items()],sep='\n')

In [97]:
model = ml.regression.DecisionTreeRegressor(featuresCol='featuresVector', labelCol='totalSales', maxBins=2048)
model = model.fit(train)

In [98]:
prediction = model.transform(train)

In [99]:
prediction.withColumn('difference', prediction['totalSales'] - prediction['prediction']).select('difference').describe().show()

+-------+--------------------+
|summary|          difference|
+-------+--------------------+
|  count|                6868|
|   mean|-8.50168314703290...|
| stddev|  1074.2186720654647|
|    min| -4899.2590363636355|
|    max|   6862.763763636364|
+-------+--------------------+



In [100]:
autoTest(report=prediction, label='totalSales', pred='prediction')

mae: 751.5942210672191
mse: 1153777.7376861675
rmse: 1074.1404645977022
r2: 0.605078252691762
var: 1767757.3399595795
